In [13]:

%load_ext autoreload
%autoreload 2

from hossam.util import *
from hossam.regression import *
from hossam.plot import *
from hossam.tensor import *
from hossam.analysis import *
from hossam.classification import *
from pandas import merge, read_excel, pivot_table, melt, to_datetime, to_timedelta, DateOffset, Timestamp, Period, Series
import numpy as np
import re
import numpy as np

# 로지스틱 회귀 통계 모형
from statsmodels.api import Logit, add_constant
# 확률밀도함수
from scipy.stats import norm
from datetime import datetime

def diff_month(d1, d2):
    return (d1.year - d2.year) * 12 + (d1.month - d2.month)

def diff_days(start_date, end_date):
    # 날짜 간 차이 계산
    delta = end_date - start_date
    # 차이를 일자로 변환하여 반환
    return delta.days

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
origin = my_read_excel("통합10-1.xlsx", info=False)
my_pretty_table(origin.head())


상위 5개 행
+----+----------------+---------------------+----------+----------+---------------------+------------+--------+--------------+------------+------------+--------+-----------+------------+--------+----------+-------------+-------------+------------+--------+------------------+------------+----------+------------+------------+------------+------------+------------+----------+----------------+----------------+--------------+----------------+------------------+------------+------------+----------+--------------+--------------+--------------+-----------+------------+-----------------+------------+------------+
|    |   회원카드번호 | 연월                | 등록명   |   개월수 | 종료시점            |   환불여부 |   양도 |   잔여개월수 |   결제시점 |   이탈여부 |   광고 |    추세값 |   환불내역 | 요일   |   미수액 |   거래일_일 |   환불일_일 |   일일입장 |   분실 |   부가서비스사용 |   레슨여부 |   매출액 |   골프전환 |   단체회원 |   골프매출 |   라카매출 |   헬스매출 |   PT매출 |   골프레슨매출 |   일일입장매출 |   양도비매출 |   필라테스매출 |   환불사용금매출 |   의류매출 |   분실매출 |   GX매출 |   상환액매출 |   입회비매출 |   

In [44]:
소비자심리지수 = my_read_excel("소비자심리지수.xlsx")
for i in range(len(소비자심리지수)):
    소비자심리지수.loc[i, '연월2']=소비자심리지수.loc[i, '연월'][:-2]
소비자심리지수.drop(['연월'], axis=1, inplace=True)
소비자심리지수['연월'] = to_datetime(소비자심리지수['연월2'])
소비자심리지수.drop(['연월2'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   연월       52 non-null     object 
 1   소비자심리지수  52 non-null     float64
dtypes: float64(1), object(1)
memory usage: 960.0+ bytes
None

상위 5개 행
+----+------------+------------------+
|    | 연월       |   소비자심리지수 |
|----+------------+------------------|
|  0 | 2020.01 월 |            105.3 |
|  1 | 2020.02 월 |               98 |
|  2 | 2020.03 월 |             79.7 |
|  3 | 2020.04 월 |             72.1 |
|  4 | 2020.05 월 |             78.8 |
+----+------------+------------------+

하위 5개 행
+----+------------+------------------+
|    | 연월       |   소비자심리지수 |
|----+------------+------------------|
| 47 | 2023.12 월 |             99.7 |
| 48 | 2024.01 월 |            101.6 |
| 49 | 2024.02 월 |            101.9 |
| 50 | 2024.03 월 |            100.7 |
| 51 | 2024.04 월 |            100.7 |
+----+------------+------------

In [45]:
문화생활비지출 = my_read_excel("문화생활비지출전망-서울.xlsx")
문화생활비지출.rename(columns={'교양·오락·문화생활비 지출전망CSI-서울': '문화생활비지출'}, inplace=True)
for i in range(len(문화생활비지출)):
    문화생활비지출.loc[i, '연월2']=문화생활비지출.loc[i, '연월'][:-2]
문화생활비지출.drop(['연월'], axis=1, inplace=True)
문화생활비지출['연월'] = to_datetime(문화생활비지출['연월2'])
문화생활비지출.drop(['연월2'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   연월                      52 non-null     object
 1   교양·오락·문화생활비 지출전망CSI-서울  52 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 960.0+ bytes
None

상위 5개 행
+----+------------+-----------------------------------------+
|    | 연월       |   교양·오락·문화생활비 지출전망CSI-서울 |
|----+------------+-----------------------------------------|
|  0 | 2020.01 월 |                                      92 |
|  1 | 2020.02 월 |                                      91 |
|  2 | 2020.03 월 |                                      79 |
|  3 | 2020.04 월 |                                      75 |
|  4 | 2020.05 월 |                                      76 |
+----+------------+-----------------------------------------+

하위 5개 행
+----+------------+-----------------------------------------+
|    | 연월     

In [46]:
CRR = my_read_excel("CRR_월별.xlsx")
CRR.drop(['Unnamed: 0'], axis=1, inplace=True)
CRR.rename(columns={'날짜':'연월'}, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Unnamed: 0  48 non-null     int64         
 1   날짜          48 non-null     datetime64[ns]
 2   CRR         47 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.2 KB
None

상위 5개 행
+----+--------------+---------------------+----------+
|    |   Unnamed: 0 | 날짜                |      CRR |
|----+--------------+---------------------+----------|
|  0 |            0 | 2020-01-01 00:00:00 |  -77.533 |
|  1 |            1 | 2020-02-01 00:00:00 | -15.1365 |
|  2 |            2 | 2020-03-01 00:00:00 | -20.2586 |
|  3 |            3 | 2020-04-01 00:00:00 | -13.9785 |
|  4 |            4 | 2020-05-01 00:00:00 | -18.3962 |
+----+--------------+---------------------+----------+

하위 5개 행
+----+--------------+---------------------+-----------+
|    |   Unnamed:

In [47]:
origin = merge(origin, 문화생활비지출, on='연월')
origin = merge(origin, 소비자심리지수, on='연월')
origin = merge(origin, CRR, on='연월')

In [48]:
origin['연월']

0      2020-09-01
1      2020-09-01
2      2020-09-01
3      2020-09-01
4      2020-09-01
          ...    
6304   2023-01-01
6305   2023-01-01
6306   2023-01-01
6307   2023-01-01
6308   2023-01-01
Name: 연월, Length: 6309, dtype: datetime64[ns]

In [49]:
origin['거래일_일']

0        8
1       18
2        8
3        8
4       25
        ..
6304    31
6305    31
6306    31
6307    31
6308    31
Name: 거래일_일, Length: 6309, dtype: int64

In [50]:
origin['연월']

0      2020-09-01
1      2020-09-01
2      2020-09-01
3      2020-09-01
4      2020-09-01
          ...    
6304   2023-01-01
6305   2023-01-01
6306   2023-01-01
6307   2023-01-01
6308   2023-01-01
Name: 연월, Length: 6309, dtype: datetime64[ns]

In [54]:
timedelta_series2.unique()

array([ -86400000000000,  604800000000000, 1814400000000000,
       1123200000000000, 1296000000000000,  777600000000000,
       1209600000000000, 1382400000000000, 1468800000000000,
        259200000000000,  518400000000000, 2592000000000000,
        172800000000000, 2419200000000000, 1036800000000000,
       2505600000000000, 2332800000000000,   86400000000000,
        345600000000000, 1641600000000000,  432000000000000,
       2246400000000000, 1555200000000000, 1900800000000000,
        864000000000000, 2073600000000000,                0,
        950400000000000, 1987200000000000, 1728000000000000,
        691200000000000, 2160000000000000], dtype='timedelta64[ns]')

In [64]:
timedelta_series = to_timedelta(origin['거래일_일'] - 1, unit='D')

timedelta_series2 = to_timedelta(origin['환불일_일'] - 1, unit='D')
origin['거래일'] = origin['연월']+timedelta_series

origin['환불일'] = origin['연월']+timedelta_series
for i in range(len(origin)):
    if origin.loc[i, '환불일_일'] == 0:
        origin.loc[i, '환불일'] =""

origin

,회원카드번호,연월,등록명,개월수,종료시점,환불여부,양도,잔여개월수,결제시점,이탈여부,...,FC 성별,트레이너,트레이너 성별,거래횟수,환불시일,문화생활비지출,소비자심리지수,CRR,거래일,환불일
0,12131325,2020-09-01,헬스,12,2021-09-01,0,0,12,0,1,...,2,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
1,13120079,2020-09-01,헬스,12,2021-11-01,0,0,14,0,1,...,0,NaN,0,1,0,77,80.7,-10.456853,2020-09-18,NaT
2,13120114,2020-09-01,헬스,12,2021-09-01,0,0,12,0,0,...,0,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
3,13120123,2020-09-01,골프,12,2021-11-01,0,0,14,0,0,...,1,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
4,13120139,2020-09-01,헬스,10,2021-07-01,0,0,10,0,0,...,1,NaN,0,1,0,77,80.7,-10.456853,2020-09-25,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,23010028,2023-01-01,헬스,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6305,23010029,2023-01-01,골프,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6306,23010030,2023-01-01,헬스,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6307,23010031,2023-01-01,골프,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT


In [67]:
origin[origin['환불일'].isna()]

,회원카드번호,연월,등록명,개월수,종료시점,환불여부,양도,잔여개월수,결제시점,이탈여부,...,FC 성별,트레이너,트레이너 성별,거래횟수,환불시일,문화생활비지출,소비자심리지수,CRR,거래일,환불일
0,12131325,2020-09-01,헬스,12,2021-09-01,0,0,12,0,1,...,2,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
1,13120079,2020-09-01,헬스,12,2021-11-01,0,0,14,0,1,...,0,NaN,0,1,0,77,80.7,-10.456853,2020-09-18,NaT
2,13120114,2020-09-01,헬스,12,2021-09-01,0,0,12,0,0,...,0,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
3,13120123,2020-09-01,골프,12,2021-11-01,0,0,14,0,0,...,1,NaN,0,1,0,77,80.7,-10.456853,2020-09-08,NaT
4,13120139,2020-09-01,헬스,10,2021-07-01,0,0,10,0,0,...,1,NaN,0,1,0,77,80.7,-10.456853,2020-09-25,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6304,23010028,2023-01-01,헬스,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6305,23010029,2023-01-01,골프,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6306,23010030,2023-01-01,헬스,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT
6307,23010031,2023-01-01,골프,12,2024-01-01,0,0,12,0,2,...,1,NaN,0,1,0,91,90.7,-10.461285,2023-01-31,NaT


In [68]:
origin['양도'].fillna(0, inplace=True)
origin['환불내역'].fillna("", inplace=True)
origin['요일'].fillna("", inplace=True) 
origin['트레이너'].fillna("", inplace=True) 
origin['환불일'].fillna("", inplace=True) 
origin['월'] = origin['연월'].dt.strftime('%m')
origin['분기'] = origin['연월'].dt.to_period(freq='Q-DEC').dt.quarter
origin['연월']=origin['연월'].dt.strftime('%Y-%m')
origin['종료시점'] =origin['종료시점'].dt.strftime('%Y-%m')
df = origin.copy()
#origin['등록명'].unique()

In [69]:
# f=df[df['이탈여부']!=2]
df.reset_index(drop=True, inplace=True)

In [72]:
df['통합매출'] = 0
for i in range(len(df)):
    df.loc[i, '통합매출'] = df.loc[i, '매출액'] + df.loc[i, '미수액'] - df.loc[i, '상환액매출']
df['매출액2'] =0
for i in range(len(df)):
    df.loc[i, '매출액2'] = df.loc[i, '매출액'] - df.loc[i, '환불사용금매출'] - df.loc[i, '양도비매출']
# df.drop(['환불사용금매출', '양도비매출','매출액'], axis=1, inplace=True)

In [73]:
df.columns

Index(['회원카드번호', '연월', '등록명', '개월수', '종료시점', '환불여부', '양도', '잔여개월수', '결제시점',
       '이탈여부', '광고', '추세값', '환불내역', '요일', '미수액', '거래일_일', '환불일_일', '일일입장',
       '분실', '부가서비스사용', '레슨여부', '매출액', '골프전환', '단체회원', '골프매출', '라카매출', '헬스매출',
       'PT매출', '골프레슨매출', '일일입장매출', '양도비매출', '필라테스매출', '환불사용금매출', '의류매출',
       '분실매출', 'GX매출', '상환액매출', '입회비매출', '예치금매출', 'FC 성별', '트레이너', '트레이너 성별',
       '거래횟수', '환불시일', '문화생활비지출', '소비자심리지수', 'CRR', '거래일', '환불일', '월', '분기',
       '통합매출', '매출액2'],
      dtype='object')

In [75]:
df.drop(['거래일_일', '환불일_일'], axis=1, inplace=True)

In [76]:
df.to_excel("통합11-2.xlsx")